# **Testing Model**

## Cek hasil serving

In [18]:
import requests
from pprint import PrettyPrinter

url = "https://diabetespredictionmlops-production.up.railway.app/v1/models/diabetes-prediction"

res = requests.get(url)

pp = PrettyPrinter()
pp.pprint(res.json())
print("\nSTATUS:", res.status_code)


{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1763528202'}]}

STATUS: 200


## Import Library

In [24]:
import tensorflow as tf
import pandas as pd
import numpy as np
import base64
import requests

## Set Variable

In [20]:
# ENDPOINT MODEL DI RAILWAY
SERVING_URL = "https://diabetespredictionmlops-production.up.railway.app/v1/models/diabetes-prediction:predict"

# TIPE FITUR RAW SESUAI SCHEMA TFX
INT_FEATURES = {"Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin", "Age", "Outcome"}
FLOAT_FEATURES = {"BMI", "DiabetesPedigreeFunction"}

## Fungsi Helper

In [21]:
def tfexample_b64_from_dict(sample: dict) -> str:
    """
    Serialize dictionary -> tf.train.Example -> base64 string.
    Sesuai input model TFX di Railway TF Serving.
    """
    def _i(v): 
        return tf.train.Feature(int64_list=tf.train.Int64List(value=[int(v)]))
    
    def _f(v): 
        return tf.train.Feature(float_list=tf.train.FloatList(value=[float(v)]))
    
    feats = {}
    for k, v in sample.items():
        if k in INT_FEATURES:
            feats[k] = _i(v)
        elif k in FLOAT_FEATURES:
            feats[k] = _f(v)
        else:
            raise ValueError(f"Fitur '{k}' tidak dikenali. Harus ada di INT_FEATURES atau FLOAT_FEATURES.")
    
    example = tf.train.Example(features=tf.train.Features(feature=feats))
    return base64.b64encode(example.SerializeToString()).decode("utf-8")


def predict_one(sample: dict, threshold: float = 0.5) -> dict:
    """
    Kirim 1 sample ke Railway TF Serving.
    Mengembalikan probability + predicted label.
    """
    payload = {"instances": [{"b64": tfexample_b64_from_dict(sample)}]}
    r = requests.post(SERVING_URL, json=payload, timeout=30)

    try:
        r.raise_for_status()
    except Exception as e:
        print("ERROR RESPONSE:", r.text)
        raise e

    prob = float(r.json()["predictions"][0][0])
    return {
        "probability": prob,
        "prediction": int(prob >= threshold)
    }

## **Testing Inference Model**

In [23]:
# Testing Inference Model berdasarkan data dari dataset
# Load dataset
try:
    df = pd.read_csv("data/diabetes.csv")
except FileNotFoundError:
    df = pd.read_csv("diabetes.csv")

# Pastikan kolom urut sesuai schema
cols = ["Pregnancies","Glucose","BloodPressure","SkinThickness","Insulin",
        "BMI","DiabetesPedigreeFunction","Age","Outcome"]
df = df[cols].copy()
LABEL = "Outcome"

# Ambil sample ke-2
idx = 2
row = df.iloc[idx]
sample_dict = row.drop(LABEL).to_dict()   # HAPUS label saat inference

print("Sample RAW dari dataset (index =", idx, "):")
for k in sample_dict:
    print(f"- {k}: {sample_dict[k]}")
print("Label ground-truth:", int(row[LABEL]))

# Prediksi via Railway
result = predict_one(sample_dict)

print("\nHASIL PREDIKSI:")
print("Probabilitas positif:", round(result["probability"], 4))
print("Prediksi:", result["prediction"])

Sample RAW dari dataset (index = 2 ):
- Pregnancies: 8.0
- Glucose: 183.0
- BloodPressure: 64.0
- SkinThickness: 0.0
- Insulin: 0.0
- BMI: 23.3
- DiabetesPedigreeFunction: 0.672
- Age: 32.0
Label ground-truth: 1

HASIL PREDIKSI:
Probabilitas positif: 0.7557
Prediksi: 1


In [30]:
# Ambil 5 sample random
subset = df.sample(5)

# Serialize seluruh sample → tf.Example base64
instances = [
    {"b64": tfexample_b64_from_dict(r.drop(LABEL).to_dict())}
    for _, r in subset.iterrows()
]

payload = {"instances": instances}

resp = requests.post(SERVING_URL, json=payload, timeout=30)
resp.raise_for_status()

preds = np.array(resp.json()["predictions"]).flatten()

results = pd.DataFrame({
    "Prob_Positive": preds,
    "Predicted_Label": (preds >= 0.5).astype(int),
    "True_Label": subset[LABEL].values
})

print(results)


   Prob_Positive  Predicted_Label  True_Label
0       0.457214                0           0
1       0.554079                1           1
2       0.225506                0           0
3       0.871454                1           1
4       0.506808                1           1


## **Inference Model (Input New Data)**

In [34]:
def input_interaktif():
    """Input data baru secara interaktif, kirim ke Railway TF Serving, dan tampilkan hasil prediksi."""
    
    print("INPUT DATA PASIEN ")

    s = {}
    s["Pregnancies"] = int(input("Pregnancies (int): "))
    s["Glucose"] = int(input("Glucose (int): "))
    s["BloodPressure"] = int(input("BloodPressure (int): "))
    s["SkinThickness"] = int(input("SkinThickness (int): "))
    s["Insulin"] = int(input("Insulin (int): "))
    s["BMI"] = float(input("BMI (float): "))
    s["DiabetesPedigreeFunction"] = float(input("DiabetesPedigreeFunction (float): "))
    s["Age"] = int(input("Age (int): "))

    print("\nDATA YANG DIINPUT:")
    for k, v in s.items():
        print(f"- {k}: {v}")

    # Kirim ke Railway
    res = predict_one(s, threshold=0.5)

    print("\nHASIL PREDIKSI MODEL:")
    print(f"Probabilitas positif diabetes : {res['probability']:.4f}")
    print(f"Prediksi akhir               : {res['prediction']} (1=positif, 0=negatif)")

In [35]:
# jalankan fungsi untuk uji dengan input manual (pasien negatif)
input_interaktif()

INPUT DATA PASIEN 

DATA YANG DIINPUT:
- Pregnancies: 1
- Glucose: 95
- BloodPressure: 70
- SkinThickness: 25
- Insulin: 80
- BMI: 23.5
- DiabetesPedigreeFunction: 0.25
- Age: 29

HASIL PREDIKSI MODEL:
Probabilitas positif diabetes : 0.0224
Prediksi akhir               : 0 (1=positif, 0=negatif)


**Pasien Non-Diabetes (Outcome = 0)**  
Karakteristik umum:
- Pregnancies: 1
- Glucose: 95 → kadar gula darah normal (< 140 mg/dL)
- BloodPressure: 70 → tekanan darah normal (sekitar 120/80 mmHg)
- SkinThickness: 25 → ketebalan kulit wajar
- Insulin: 80 → kadar insulin dalam batas normal
- BMI: 23.5 → berat badan ideal (18.5–24.9)
- DiabetesPedigreeFunction: 0.25 → faktor genetik rendah
- Age: 29 → usia relatif muda

**Interpretasi:** Kemungkinan besar tidak diabetes karena kadar glukosa dan BMI masih dalam batas normal.

In [36]:
# jalankan fungsi untuk uji dengan input manual (pasien positif)
input_interaktif()

INPUT DATA PASIEN 

DATA YANG DIINPUT:
- Pregnancies: 6
- Glucose: 165
- BloodPressure: 82
- SkinThickness: 35
- Insulin: 130
- BMI: 35.2
- DiabetesPedigreeFunction: 0.78
- Age: 50

HASIL PREDIKSI MODEL:
Probabilitas positif diabetes : 0.7929
Prediksi akhir               : 1 (1=positif, 0=negatif)


**Pasien Diabetes (Outcome = 1)**  
Karakteristik umum:
- Pregnancies: 6
- Glucose: 165 → kadar glukosa tinggi (≥ 140 mg/dL indikatif diabetes)
- BloodPressure: 82 → sedikit di atas normal
- SkinThickness: 35 → cenderung tinggi (indikasi lemak subkutan lebih banyak)
- Insulin: 130 → kadar insulin tinggi
- BMI: 35.2 → obesitas (BMI ≥ 30)
- DiabetesPedigreeFunction: 0.78 → faktor genetik sedang hingga tinggi
- Age: 50 → termasuk kategori risiko tinggi

**Interpretasi:** Kombinasi glukosa tinggi, obesitas, dan usia lanjut mengindikasikan diabetes.